MNIST logistic regression ( Code is heavily inspired by book TensorFlow 1.x Deep Learning Cookbook )

In [123]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.examples.tutorials.mnist import input_data

def get_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)
    return mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
    
class LogisticRegressionModel():
    '''
    Logistic Regression Model
    '''
    
    def __init__(self, learning_rate = 0.01, shape = (784, 10)):
        self.learning_rate = learning_rate
        self.shape = shape
        # placeholders
        self.X = tf.placeholder(tf.float32, name = 'X')
        self.Y = tf.placeholder(tf.float32, name = 'Y')
        #variables
        self.w = tf.Variable(tf.random_normal([self.shape[0],self.shape[1]]), name = 'w')
        self.b = tf.Variable(tf.zeros([self.shape[1]]), name = 'bias')
        #predicted labels
        self.Y_hat = tf.nn.softmax(tf.matmul(self.X, self.w)+ self.b)
        #loss and optimization
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.Y, logits = self.Y_hat, name = 'loss'))
        self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss)
        # prediction and accuracy
        self.correct_prediction = tf.equal(tf.argmax(self.Y_hat, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        # for tensorboard
        self.w_histogram = tf.summary.histogram('weights', self.w)
        self.b_histrogram = tf.summary.histogram('biases', self.b)
        self.loss_scalar = tf.summary.scalar('cross-entropy', self.loss)
        self.accuracy_scalar = tf.summary.scalar('accuracy', self.accuracy)
        self.merged_summary_op = tf.summary.merge_all()
        # inititialization and session stuff
        self.init_op = tf.global_variables_initializer()
        self.sess = tf.Session()
        
    def fit(self, X, Y, epochs = 100, every = 10, folder = 'LogisticRegression', batch_size = 500, init = True ):
        '''
        optimizes weights
        args:
          X: <numpy.array>, features 
          Y: <numpy.array>, labels
          epochs: <int>, number of epochs
          every: <int>, how often print message with Epoch and Loss values
          folder: <string>, name of folder where to store data for TensorBoard
          batch_size: <int>, size of features array, size of features sample
        '''
        total = []
        if init:
            self.sess.run(self.init_op)
        summary_writer = tf.summary.FileWriter(folder, self.sess.graph)
        x_length = len(X)
        for i in range(epochs):
            batch = np.random.randint(0, x_length - batch_size, 1)[0]
            x_batch = X[batch:batch + batch_size]
            y_batch = Y[batch:batch + batch_size]
            _, l, a = self.sess.run([self.optimizer, self.loss, self.accuracy], feed_dict = {self.X: x_batch, self.Y: y_batch})
            w = self.sess.run(self.w_histogram)
            b = self.sess.run(self.b_histrogram)
            loss, accuracy = self.sess.run([self.loss_scalar, self.accuracy_scalar ], feed_dict = {self.X: x_batch, self.Y: y_batch})
            summary_writer.add_summary(w, i)
            summary_writer.add_summary(b, i)
            summary_writer.add_summary(loss, i)
            summary_writer.add_summary(accuracy)
            total.append(l)
            if i % every == 0:
                print('Epoch {} Loss: {} Accuracy {}'.format(i, l, a))
        weights = self.sess.run(self.w)
        return total, weights
    
    def predict(self, X, before_fit = False):
        '''
        return predicted values, 
        
        args:
          X: <numpy.array>, features
          before_fit: <boolean>, change to True if you want to use before calling fit method
        '''
        if before_fit:
            self.sess.run(self.init_op)
        Y_hat = self.sess.run(self.Y_hat, feed_dict = {self.X: X})
        return Y_hat
    
    def show(self, values):
        '''
        plot graph
        
        args:
          values: <list>, list or array of values to be plotted
        '''
        plt.plot(values)
        plt.show()
    
    def close_session(self):
        '''
        closes tensorflow session
        '''
        self.sess.close()
        return True
    
    def close_and_reset(self):
        '''
        closes tensorflow session, clears the default graph stack and resets the global default graph.
        '''
        self.sess.close()
        tf.reset_default_graph()
        return True

In [18]:
X_train, Y_train, X_test, Y_test = get_mnist()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
log = LogisticRegressionModel()
losses, weights = log.fit(X_train, Y_train, epochs = 500000, every = 10000, batch_size = 1500)

Epoch 0 Loss: 2.345693588256836 Accuracy 0.10333333164453506
Epoch 10000 Loss: 2.273240804672241 Accuracy 0.17866666615009308
Epoch 20000 Loss: 2.2020998001098633 Accuracy 0.2526666522026062
Epoch 30000 Loss: 2.140986919403076 Accuracy 0.3166666626930237
Epoch 40000 Loss: 2.141397714614868 Accuracy 0.3160000145435333
Epoch 50000 Loss: 2.1065547466278076 Accuracy 0.34933334589004517
Epoch 60000 Loss: 2.1046860218048096 Accuracy 0.3526666760444641
Epoch 70000 Loss: 2.1148412227630615 Accuracy 0.3440000116825104
Epoch 80000 Loss: 2.1084766387939453 Accuracy 0.3479999899864197
Epoch 90000 Loss: 2.1058738231658936 Accuracy 0.3526666760444641
Epoch 100000 Loss: 2.0892679691314697 Accuracy 0.36666667461395264
Epoch 110000 Loss: 2.0803966522216797 Accuracy 0.3766666650772095
Epoch 120000 Loss: 2.08878231048584 Accuracy 0.36800000071525574
Epoch 130000 Loss: 2.0986998081207275 Accuracy 0.36000001430511475
Epoch 140000 Loss: 2.0744729042053223 Accuracy 0.3813333213329315
Epoch 150000 Loss: 2.074

In [109]:
'''
below short numpy code as calculate accuracy for this model
'''
predict = log.predict(X_test[:20]) #predicted values
predicted = [np.argmax(item) for item in predict] # change from one_hot encoding to numerical values
target = [np.argmax(item) for item in Y_test[:20]] # change from one_hot encoding to numerical values
comparison = np.equal(predicted, target).astype(int) # returns boolean comparison converted to integers (0,1)
sum(comparison)/len(comparison) # sum of ones divided by length of array

In [110]:
target = [np.argmax(item) for item in Y_test[:20]]

In [99]:
log.close_and_reset()

True

In [117]:
comparison = np.equal(predicted, target).astype(int)
sum(comparison)/len(comparison)

0.7